In [14]:
pip install dash


  Obtaining dependency information for dash from https://files.pythonhosted.org/packages/7b/62/438626ab498869249e99376684ba6d88931e7c2dd86d03a92d9d7a9b4c54/dash-2.14.2-py3-none-any.whl.metadata
  Using cached dash-2.14.2-py3-none-any.whl.metadata (11 kB)
  Using cached dash_html_components-2.0.0-py3-none-any.whl (4.1 kB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl (3.8 kB)
  Using cached dash_table-5.0.0-py3-none-any.whl (3.9 kB)
  Using cached retrying-1.3.4-py3-none-any.whl (11 kB)
  Obtaining dependency information for ansi2html from https://files.pythonhosted.org/packages/42/d7/1bc3433a2406b891182c25a7926abe3eb19af5870d540375ebf7795885b3/ansi2html-1.9.1-py3-none-any.whl.metadata
  Using cached ansi2html-1.9.1-py3-none-any.whl.metadata (3.7 kB)
Using cached dash-2.14.2-py3-none-any.whl (10.2 MB)
Using cached ansi2html-1.9.1-py3-none-any.whl (17 kB)


In [15]:
import os
import numpy as np
import pandas as pd

# import dash
# from dash import html, dcc, Input, Output, State, callback
# from dash.exceptions import PreventUpdate

import plotly.express as px
import xgboost as xgb
import pickle

# with open('xgb_model.pkl', 'rb') as file:
#     modelo_xgb = pickle.load(file)
# from dash import html

# from .templates.kpi import generate_kpi

# dash.register_page(__name__, path='/predict')

root_path = r'C:\Users\Josue\Documentos\Maestría\Aix-Marseille\SQL\Dash project\src\data\data.csv'

try:
    df = pd.read_csv(root_path)
except FileNotFoundError:
    print(f"Error! The file 'data.csv' was not found at the provided path: {root_path}")

# Convert 'BMI' column to numeric and replace non-numeric values with NaN
df['BMI'] = pd.to_numeric(df['BMI'], errors='coerce')

# Drop rows with missing values
df.dropna(inplace=True)

# Check for remaining missing values
if df.isnull().values.any():
    print("Warning: There are still missing values in the dataset after dropping them. Check the data.")

# Check for remaining non-numeric values in the 'BMI' column
# non_numeric_values_bmi = [df.apply(lambda x: not pd.api.types.is_numeric_dtype(x))].unique()
# if non_numeric_values_bmi.any():
#     print(f"Non-numeric values in column 'BMI': {non_numeric_values_bmi}")

# Optionally, fill remaining missing values with a specific value
# df.fillna(0, inplace=True)  # Replace NaN with 0 or any other value as needed


modelo_xgb = xgb.Booster()
modelo_xgb.load_model(r'C:\Users\Josue\Documentos\Maestría\Aix-Marseille\SQL\Dash project\src\models\xgb.model')

cols = ['BMI', 'HighChol', 'HeartDiseaseorAttack', 'PhysHlth', 'Sex']

options_features = [{'label': col, 'value': col} for col in cols]



# # Convert object-type features to numeric
# numeric_cols = ['BMI', 'HighChol', 'HeartDiseaseorAttack', 'PhysHlth']

# for col in numeric_cols:
#     df[col] = pd.to_numeric(df[col], errors='coerce')

# # Check for non-numeric values in the columns
# for col in numeric_cols:
#     non_numeric_values = df[col][df[col].apply(lambda x: not pd.api.types.is_numeric_dtype(x))].unique()
#     if non_numeric_values.any():
#         print(f"Non-numeric values in column '{col}': {non_numeric_values}")

# # Ensure that all columns are numeric
# df = df.apply(pd.to_numeric, errors='coerce')

# # Check for missing values
# if df.isnull().values.any():
#     print("Warning: There are missing values in the dataset. Consider handling or imputing them.")



# List of features that should have binary sliders
binary_features = ['HighChol', 'HeartDiseaseorAttack', 'Sex']



# Process selected features and sliders to update the model accordingly
# You can use selected_features list and corresponding sliders' values in your prediction logic

# Example:
features_to_include = ['BMI', 'HighChol', 'HeartDiseaseorAttack', 'PhysHlth']
# selected_features = [feature for feature in features_to_include if feature in selected_features]

# Access slider values using the triggered_id
# sliders_values = {feature: dash.callback_context.triggered_id.split('.')[0] for feature in selected_features}
# Assuming you have already defined input_data_dict earlier in your code
input_data_dict = {'Sex': 0, 'BMI': 18, 'HighChol': 1, 'HeartDiseaseorAttack': 1, 'PhysHlth': 30}  # Add other variables as needed

# Perform the prediction using the updated features and sliders values
input_data = [0] * len(features_to_include)  # Assuming other features are 0 for now
for feature in features_to_include:
    # Access values from input_data_dict using the feature name
    input_data[features_to_include.index(feature)] = input_data_dict[feature]

# The rest of your code remains the same...
  # Add other variables as needed
# Perform the prediction using the updated features and sliders values
input_data = [0] * len(features_to_include)  # Assuming other features are 0 for now
for feature in features_to_include:
    input_data[features_to_include.index(feature)] = input_data_dict[feature]


input_data_dict.update({feature: value for feature, value in zip(features_to_include, input_data)})

for key, value in input_data_dict.items():
    if not isinstance(value, (int, float)):
        try:
            # Try to convert to float
            input_data_dict[key] = float(value)
        except ValueError:
            # If conversion fails, print a message and handle it as per your requirement
            print(f"Cannot convert value for {key} to a number.")
# print(input_data_dict, index=[0])
# Use DMatrix with enable_categorical=True
dmatrix = xgb.DMatrix(pd.DataFrame(input_data_dict, index=[0]), enable_categorical=True)
y_pred = int(modelo_xgb.predict(dmatrix))

if y_pred == 1:
    prediction_message = "Probability of having Diabetes"
else:
    prediction_message = "You are healthy. No probabilities of having Diabetes"
print(prediction_message)
# kpi1 = generate_kpi(prediction_message, f'{y_pred}%')

# kpis_output = [kpi1]
# print(kpis_output)


ImportError: attempted relative import with no known parent package